In [35]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import pandas as pd
import os

In [37]:
# Define the path to the CSV file
csv_file_path = r'E:\DS440proj\BTCpred\volume\data\raw\Macro\raw_data.csv'

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Display the first few rows of the dataframe
print(df.head())

                    Datetime  SP500  DOW30  EuroStoxx50  NASDAQ   CrudeOil  \
0  2022-02-27 18:00:00-05:00    NaN    NaN          NaN     NaN  96.760002   
1  2022-02-27 19:00:00-05:00    NaN    NaN          NaN     NaN  96.580002   
2  2022-02-27 20:00:00-05:00    NaN    NaN          NaN     NaN  96.470001   
3  2022-02-28 09:30:00+08:00    NaN    NaN          NaN     NaN        NaN   
4  2022-02-27 21:00:00-05:00    NaN    NaN          NaN     NaN  96.120003   

           SSE         Gold  VIX     Nikkei225  FTSE100    EURUSD    CNYUSD  \
0          NaN  1912.500000  NaN           NaN      NaN       NaN       NaN   
1          NaN  1919.900024  NaN  26395.560547      NaN  1.118443  0.158306   
2          NaN  1915.400024  NaN  26420.269531      NaN  1.117818  0.158341   
3  3445.742432          NaN  NaN           NaN      NaN       NaN       NaN   
4          NaN  1911.599976  NaN  26393.419922      NaN  1.116071  0.158411   

     GBPUSD    JPYUSD    CHFUSD  
0       NaN       NaN 

In [38]:
def fill_single_interval_missings(dataframe, columns):
    for column in columns:
        # Loop through each row except the first and last
        for i in range(1, len(dataframe) - 1):
            # Check if the current value is NaN and neighbors are not
            if pd.isna(dataframe.at[dataframe.index[i], column]) and \
               not pd.isna(dataframe.at[dataframe.index[i - 1], column]) and \
               not pd.isna(dataframe.at[dataframe.index[i + 1], column]):
                # Calculate the midpoint of its neighbors
                midpoint = (dataframe.at[dataframe.index[i - 1], column] + dataframe.at[dataframe.index[i + 1], column]) / 2
                # Replace the NaN with the midpoint
                dataframe.at[dataframe.index[i], column] = midpoint
    return dataframe

# Replace single-interval missing values for specified columns
df = fill_single_interval_missings(df, ['SP500', 'DOW30', 'NASDAQ','SSE'])


In [39]:
df

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
0,2022-02-27 18:00:00-05:00,NaN,NaN,NaN,NaN,96.760002,NaN,1912.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-27 19:00:00-05:00,NaN,NaN,NaN,NaN,96.580002,NaN,1919.900024,NaN,26395.560547,NaN,1.118443,0.158306,1.336863,0.008652,1.079436
2,2022-02-27 20:00:00-05:00,NaN,NaN,NaN,NaN,96.470001,NaN,1915.400024,NaN,26420.269531,NaN,1.117818,0.158341,1.336041,0.008657,1.078981
3,2022-02-28 09:30:00+08:00,NaN,NaN,NaN,NaN,NaN,3445.742432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-02-27 21:00:00-05:00,NaN,NaN,NaN,NaN,96.120003,3446.172607,1911.599976,NaN,26393.419922,NaN,1.116071,0.158411,1.335738,0.008659,1.079063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18924,2024-02-26 15:00:00-05:00,5073.949951,39093.189453,NaN,15988.945801,77.570000,NaN,2042.099976,13.73,NaN,NaN,1.085423,0.138955,1.268553,0.006637,1.136260
18925,2024-02-26 15:30:00-05:00,5069.529785,39069.230469,NaN,15976.250977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18926,2024-02-26 16:00:00-05:00,NaN,NaN,NaN,NaN,77.599998,NaN,2040.900024,13.74,NaN,NaN,1.085423,0.138955,1.268601,0.006636,1.136441
18927,2024-02-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.085305,0.138955,1.268376,0.006638,1.136699


In [40]:
from pytz import timezone

# Convert 'Datetime' to datetime object and set timezone to UTC since original times are in +08:00
df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)

# Convert 'Datetime' from UTC to Eastern Time (considering Eastern Standard Time without daylight saving)
df['Datetime'] = df['Datetime'].dt.tz_convert('EST')

# Remove rows with half-hour times
df = df[~df['Datetime'].dt.minute.isin([30])]

# Display the first few rows of the modified dataframe
df.head()

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
0,2022-02-27 18:00:00-05:00,NaN,NaN,NaN,NaN,96.760002,NaN,1912.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-27 19:00:00-05:00,NaN,NaN,NaN,NaN,96.580002,NaN,1919.900024,NaN,26395.560547,NaN,1.118443,0.158306,1.336863,0.008652,1.079436
2,2022-02-27 20:00:00-05:00,NaN,NaN,NaN,NaN,96.470001,NaN,1915.400024,NaN,26420.269531,NaN,1.117818,0.158341,1.336041,0.008657,1.078981
4,2022-02-27 21:00:00-05:00,NaN,NaN,NaN,NaN,96.120003,3446.172607,1911.599976,NaN,26393.419922,NaN,1.116071,0.158411,1.335738,0.008659,1.079063
6,2022-02-27 22:00:00-05:00,NaN,NaN,NaN,NaN,96.739998,3446.522949,1909.699951,NaN,26347.970703,NaN,1.115449,0.158541,1.334793,0.008657,1.078737


In [41]:
data_filled = df.fillna(method='ffill')

# Display the first few rows of the dataframe to confirm changes
data_filled.head()

C:\Users\13275\AppData\Local\Temp\ipykernel_59004\1185419078.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_filled = df.fillna(method='ffill')


,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
0,2022-02-27 18:00:00-05:00,NaN,NaN,NaN,NaN,96.760002,NaN,1912.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-27 19:00:00-05:00,NaN,NaN,NaN,NaN,96.580002,NaN,1919.900024,NaN,26395.560547,NaN,1.118443,0.158306,1.336863,0.008652,1.079436
2,2022-02-27 20:00:00-05:00,NaN,NaN,NaN,NaN,96.470001,NaN,1915.400024,NaN,26420.269531,NaN,1.117818,0.158341,1.336041,0.008657,1.078981
4,2022-02-27 21:00:00-05:00,NaN,NaN,NaN,NaN,96.120003,3446.172607,1911.599976,NaN,26393.419922,NaN,1.116071,0.158411,1.335738,0.008659,1.079063
6,2022-02-27 22:00:00-05:00,NaN,NaN,NaN,NaN,96.739998,3446.522949,1909.699951,NaN,26347.970703,NaN,1.115449,0.158541,1.334793,0.008657,1.078737


In [42]:
data_filled

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
0,2022-02-27 18:00:00-05:00,NaN,NaN,NaN,NaN,96.760002,NaN,1912.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-27 19:00:00-05:00,NaN,NaN,NaN,NaN,96.580002,NaN,1919.900024,NaN,26395.560547,NaN,1.118443,0.158306,1.336863,0.008652,1.079436
2,2022-02-27 20:00:00-05:00,NaN,NaN,NaN,NaN,96.470001,NaN,1915.400024,NaN,26420.269531,NaN,1.117818,0.158341,1.336041,0.008657,1.078981
4,2022-02-27 21:00:00-05:00,NaN,NaN,NaN,NaN,96.120003,3446.172607,1911.599976,NaN,26393.419922,NaN,1.116071,0.158411,1.335738,0.008659,1.079063
6,2022-02-27 22:00:00-05:00,NaN,NaN,NaN,NaN,96.739998,3446.522949,1909.699951,NaN,26347.970703,NaN,1.115449,0.158541,1.334793,0.008657,1.078737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2024-02-26 14:00:00-05:00,5079.910156,39118.304688,4864.290039,16011.191895,77.709999,2975.510132,2039.599976,13.77,39198.460938,7684.299805,1.084952,0.138955,1.267813,0.006633,1.135241
18924,2024-02-26 15:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.570000,2975.510132,2042.099976,13.73,39198.460938,7684.299805,1.085423,0.138955,1.268553,0.006637,1.136260
18926,2024-02-26 16:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.599998,2975.510132,2040.900024,13.74,39198.460938,7684.299805,1.085423,0.138955,1.268601,0.006636,1.136441
18927,2024-02-26 17:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.599998,2975.510132,2040.900024,13.74,39198.460938,7684.299805,1.085305,0.138955,1.268376,0.006638,1.136699


In [43]:
df = data_filled.dropna()

In [44]:
df

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,FTSE100,EURUSD,CNYUSD,GBPUSD,JPYUSD,CHFUSD
23,2022-02-28 10:00:00-05:00,4360.924805,33767.826172,3915.540039,13709.718262,95.510002,3457.902100,1910.400024,30.559999,26522.330078,7435.509766,1.123469,0.158519,1.341814,0.008683,1.089978
25,2022-02-28 11:00:00-05:00,4369.179932,33849.894531,3933.479980,13734.242676,95.440002,3457.902100,1894.500000,28.690001,26522.330078,7465.209961,1.122460,0.158519,1.341832,0.008687,1.089514
27,2022-02-28 12:00:00-05:00,4349.699951,33720.744141,3933.479980,13666.251465,96.209999,3457.902100,1900.900024,30.639999,26522.330078,7465.209961,1.120699,0.158519,1.339011,0.008687,1.088459
29,2022-02-28 13:00:00-05:00,4332.359863,33589.654297,3933.479980,13607.824707,96.459999,3457.902100,1900.699951,30.820000,26522.330078,7465.209961,1.121579,0.158519,1.341058,0.008699,1.090215
31,2022-02-28 14:00:00-05:00,4331.229980,33588.664062,3933.479980,13605.203125,95.680000,3457.902100,1908.300049,31.920000,26522.330078,7465.209961,1.122586,0.158519,1.341886,0.008706,1.091227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2024-02-26 14:00:00-05:00,5079.910156,39118.304688,4864.290039,16011.191895,77.709999,2975.510132,2039.599976,13.770000,39198.460938,7684.299805,1.084952,0.138955,1.267813,0.006633,1.135241
18924,2024-02-26 15:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.570000,2975.510132,2042.099976,13.730000,39198.460938,7684.299805,1.085423,0.138955,1.268553,0.006637,1.136260
18926,2024-02-26 16:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.599998,2975.510132,2040.900024,13.740000,39198.460938,7684.299805,1.085423,0.138955,1.268601,0.006636,1.136441
18927,2024-02-26 17:00:00-05:00,5073.949951,39093.189453,4864.290039,15988.945801,77.599998,2975.510132,2040.900024,13.740000,39198.460938,7684.299805,1.085305,0.138955,1.268376,0.006638,1.136699


In [45]:
df.to_csv("processed_macro_data.csv")

In [46]:
import pandas as pd

# Load the first CSV
df1 = pd.read_csv('processed_macro_data.csv', parse_dates=['Datetime'])

# Load the second CSV
df2 = pd.read_csv('merged_volume_data.csv', parse_dates=['time'])
df2.rename(columns={'time': 'Datetime'}, inplace=True)
# Convert the timezone-naive column in df2 to timezone-aware, matching df1's timezone
df2['Datetime'] = df2['Datetime'].dt.tz_localize('UTC-05:00')

# Now you can merge as before
merged_df = pd.merge(df, df2, on='Datetime', how='outer')
merged_df.sort_values(by='Datetime', inplace=True)

In [47]:
merged_df = merged_df.dropna()
merged_df.to_csv('processed_hourly_data.csv')

In [48]:
merged_df

,Datetime,SP500,DOW30,EuroStoxx50,NASDAQ,CrudeOil,SSE,Gold,VIX,Nikkei225,...,total_volume_quote,total_volume_base,total_volume_total,volume,high,low,open,close,volumefrom,volumeto
667,2022-02-28 10:00:00-05:00,4360.924805,33767.826172,3915.540039,13709.718262,95.510002,3457.902100,1910.400024,30.559999,26522.330078,...,4.074564e+08,8.501518e+09,8.908974e+09,100298.59,40917.71,39652.14,39849.29,40845.79,7495.87,3.026021e+08
668,2022-02-28 11:00:00-05:00,4369.179932,33849.894531,3933.479980,13734.242676,95.440002,3457.902100,1894.500000,28.690001,26522.330078,...,2.951618e+08,7.640820e+09,7.935982e+09,71235.25,41338.07,40736.36,40845.79,41021.81,5232.77,2.148198e+08
669,2022-02-28 12:00:00-05:00,4349.699951,33720.744141,3933.479980,13666.251465,96.209999,3457.902100,1900.900024,30.639999,26522.330078,...,2.022620e+08,6.928419e+09,7.130681e+09,40264.68,41320.14,40768.17,41021.81,41319.61,2842.60,1.165622e+08
670,2022-02-28 13:00:00-05:00,4332.359863,33589.654297,3933.479980,13607.824707,96.459999,3457.902100,1900.699951,30.820000,26522.330078,...,1.731512e+08,6.676633e+09,6.849784e+09,34319.26,41427.99,41082.37,41319.61,41217.44,2455.10,1.013162e+08
671,2022-02-28 14:00:00-05:00,4331.229980,33588.664062,3933.479980,13605.203125,95.680000,3457.902100,1908.300049,31.920000,26522.330078,...,1.655895e+08,5.810772e+09,5.976362e+09,28324.04,41377.67,41057.84,41217.44,41357.77,2007.00,8.271987e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18064,2024-02-20 11:00:00-05:00,4969.645020,38550.503906,4763.259766,15567.405273,78.860001,2920.561279,2038.500000,15.580000,38409.390625,...,1.727552e+09,1.730651e+09,3.458204e+09,33578.32,51795.47,51369.97,51693.91,51396.39,1748.02,9.020375e+07
18065,2024-02-20 12:00:00-05:00,4961.074951,38504.060547,4763.259766,15532.355957,79.169998,2920.561279,2040.000000,15.640000,38409.390625,...,3.983693e+08,3.721585e+09,4.119954e+09,44409.48,51397.71,50766.77,51396.39,51321.90,3702.71,1.892794e+08
18066,2024-02-20 13:00:00-05:00,4965.669922,38540.031250,4763.259766,15555.064941,79.230003,2920.561279,2038.599976,15.500000,38409.390625,...,1.272772e+08,1.996156e+09,2.123433e+09,22321.76,51536.57,51236.14,51321.90,51504.22,1518.03,7.796947e+07
18067,2024-02-20 14:00:00-05:00,4973.215088,38575.441406,4763.259766,15592.756348,78.269997,2920.561279,2036.500000,15.330000,38409.390625,...,2.904525e+08,2.182250e+09,2.472703e+09,28053.63,52111.16,51439.74,51504.22,52095.65,1791.37,9.274488e+07
